In [5]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.0.6816


In [1]:
pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 32.6 MB/s eta 0:00:00


In [4]:
import cassandra
print(cassandra.__version__)


3.29.2


In [17]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider


cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")






# Step 2: Load the CSV data using pandas
csv_url = 'sales_100.csv'
data = pd.read_csv(csv_url)

# Step 3: Insert data into the Bronze table
insert_query = """
    INSERT INTO bronze_sales (Region, Country, Item_Type, Sales_Channel, Order_Priority, Order_Date,
                              Order_ID, Ship_Date, Units_Sold, Unit_Price, Unit_Cost, Total_Revenue,
                              Total_Cost, Total_Profit)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Loop through the dataframe and insert each row into the Cassandra table
for _, row in data.iterrows():
    session.execute(insert_query, (row['Region'], row['Country'], row['Item Type'], row['Sales Channel'],
                                   row['Order Priority'], row['Order Date'], row['Order ID'], row['Ship Date'],
                                   row['UnitsSold'], row['UnitPrice'], row['UnitCost'], row['TotalRevenue'],
                                   row['TotalCost'], row['TotalProfit']))

print("Data inserted into the Bronze table.")


4.0.0.6816


InvalidRequest: Error from server: code=2200 [Invalid query] message="Unable to coerce '7/27/2012' to a formatted date (long)"

In [18]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from datetime import datetime

cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

# Step 2: Load the CSV data using pandas
csv_url = 'sales_100.csv'
data = pd.read_csv(csv_url)

# Step 3: Convert the 'Order Date' and 'Ship Date' columns to the correct format (YYYY-MM-DD)
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

# Step 4: Prepare the insert query
insert_query = """
    INSERT INTO bronze_sales (Region, Country, Item_Type, Sales_Channel, Order_Priority, Order_Date,
                              Order_ID, Ship_Date, Units_Sold, Unit_Price, Unit_Cost, Total_Revenue,
                              Total_Cost, Total_Profit)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Step 5: Loop through the dataframe and insert each row into the Cassandra table
for _, row in data.iterrows():
    session.execute(insert_query, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], row['Order ID'], row['Ship Date'], row['UnitsSold'], row['UnitPrice'],
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))

print("Data inserted into the Bronze table.")


4.0.0.6816
Data inserted into the Bronze table.


In [26]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from datetime import datetime

cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")


# Step 2: Load the data from the Bronze table into a pandas DataFrame
select_query = "SELECT * FROM bronze_sales"
rows = session.execute(select_query)
data = pd.DataFrame(list(rows))

# Step 3: Inspect and clean the column names
print("Column names in the DataFrame:", data.columns)
data.columns = data.columns.str.strip()  # Strip leading/trailing spaces
print("Cleaned column names:", data.columns)

# Step 4: Data Cleaning and Transformation
# Convert the 'Order_Date' and 'Ship_Date' columns to proper date format
data['order_date'] = pd.to_datetime(data['order_date'], format='%Y-%m-%d', errors='coerce')
data['ship_date'] = pd.to_datetime(data['ship_date'], format='%Y-%m-%d', errors='coerce')

# Drop rows with missing or invalid values in critical columns
data.dropna(subset=['order_date', 'ship_date', 'units_sold', 'unit_price', 'unit_cost'], inplace=True)

# Step 5: Add the 'ProfitMargin' derived column (Profit / Revenue)
data['profitmargin'] = data['total_profit'] / data['total_revenue']

# Step 6: Prepare the insert query for the Silver table
insert_query = """
    INSERT INTO silver_sales (region, country, item_type, sales_channel, order_priority, order_date,
                              order_id, ship_date, units_sold, unit_price, unit_cost, total_revenue,
                              total_cost, total_profit, profitmargin)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Step 7: Loop through the dataframe and insert each row into the Silver table
for _, row in data.iterrows():
    session.execute(insert_query, (
        row['region'], row['country'], row['item_type'], row['sales_channel'], row['order_priority'],
        row['order_date'].strftime('%Y-%m-%d'), row['order_id'], row['ship_date'].strftime('%Y-%m-%d'),
        row['units_sold'], row['unit_price'], row['unit_cost'], row['total_revenue'], row['total_cost'],
        row['total_profit'], row['profitmargin']  # Insert ProfitMargin here
    ))

print("Data inserted into the Silver table.")


4.0.0.6816
Column names in the DataFrame: Index(['order_id', 'country', 'item_type', 'order_date', 'order_priority',
       'region', 'sales_channel', 'ship_date', 'total_cost', 'total_profit',
       'total_revenue', 'unit_cost', 'unit_price', 'units_sold'],
      dtype='object')
Cleaned column names: Index(['order_id', 'country', 'item_type', 'order_date', 'order_priority',
       'region', 'sales_channel', 'ship_date', 'total_cost', 'total_profit',
       'total_revenue', 'unit_cost', 'unit_price', 'units_sold'],
      dtype='object')
Data inserted into the Silver table.


In [37]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Establish a connection to your Astra Cassandra database
cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

# Fetch data from silver_sales table
query = "SELECT * FROM silver_sales"
rows = session.execute(query)
df = pd.DataFrame(rows)

# Aggregating data for gold_sales_region table
gold_region_data = df.groupby('region').agg(
    Total_Revenue=('total_revenue', 'sum'),
    Total_Cost=('total_cost', 'sum'),
    Total_Profit=('total_profit', 'sum')
).reset_index()

# Inserting aggregated data into the gold_sales_region table
for _, row in gold_region_data.iterrows():
    insert_query = """
    INSERT INTO gold_sales_region (region, total_revenue, total_cost, total_profit)
    VALUES (%s, %s, %s, %s)
    """
    session.execute(insert_query, (row['region'], row['total_revenue'], row['total_cost'], row['total_profit']))

# Aggregating data for gold_sales_item_type table
gold_item_type_data = df.groupby('item_type').agg(
    Total_Units_Sold=('units_sold', 'sum'),
    Total_Revenue=('total_revenue', 'sum'),
    Total_Profit=('total_profit', 'sum')
).reset_index()

# Inserting aggregated data into the gold_sales_item_type table
for _, row in gold_item_type_data.iterrows():
    insert_query = """
    INSERT INTO gold_sales_item_type (item_type, total_units_sold, total_revenue, total_profit)
    VALUES (%s, %s, %s, %s)
    """
    session.execute(insert_query, (row['item_type'], row['total_units_sold'], row['total_revenue'], row['total_profit']))

# Aggregating data for gold_sales_country table
gold_country_data = df.groupby('country').agg(
    Total_Revenue=('total_revenue', 'sum'),
    Total_Profit=('total_profit', 'sum')
).reset_index()

# Inserting aggregated data into the gold_sales_country table
for _, row in gold_country_data.iterrows():
    insert_query = """
    INSERT INTO gold_sales_country (country, total_revenue, total_profit)
    VALUES (%s, %s, %s)
    """
    session.execute(insert_query, (row['country'], row['total_revenue'], row['total_profit']))

print("Data successfully inserted into gold tables.")


4.0.0.6816


KeyError: 'total_revenue'

In [41]:
import csv
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement

cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

# Define the path to the CSV file
csv_file_path = 'sales_100.csv'  # Change this to your actual file path

# Prepare the insert statement
insert_statement = """
    INSERT INTO gold_sales_region (region, total_cost, total_profit, total_revenue)
    VALUES (?, ?, ?, ?)
"""

# Use a prepared statement for better performance and security
prepared = session.prepare(insert_statement)

# Open the CSV file and read the data
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        # Prepare data tuple for insertion
        region = row['Region']
        total_cost = float(row['TotalCost'])
        total_profit = float(row['TotalProfit'])
        total_revenue = float(row['TotalRevenue'])

        # Execute the insert statement
        session.execute(prepared, (region, total_cost, total_profit, total_revenue))

# Close the session and connection
cluster.shutdown()

print("Data inserted successfully into gold_sales_region.")


4.0.0.6816
Data inserted successfully into gold_sales_region.


In [42]:
import csv
from cassandra.cluster import Cluster

cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

# Define the path to the CSV file
csv_file_path = 'sales_100.csv'  # Change this to your actual file path

# Prepare the insert statement with '?' placeholders
insert_statement = """
    INSERT INTO gold_sales_item_type (item_type, total_profit, total_revenue)
    VALUES (?, ?, ?)
"""

# Use a prepared statement for better performance and security
prepared = session.prepare(insert_statement)

# Open the CSV file and read the data
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        # Extract relevant columns from the CSV
        item_type = row['Item Type']
        total_profit = float(row['TotalProfit'])
        total_revenue = float(row['TotalRevenue'])

        # Execute the insert statement
        session.execute(prepared, (item_type, total_profit, total_revenue))

# Close the session and connection
cluster.shutdown()

print("Data inserted successfully into gold_sales_item_type.")


4.0.0.6816
Data inserted successfully into gold_sales_item_type.


In [43]:
import csv
from cassandra.cluster import Cluster

cloud_config= {
  'secure_connect_bundle': 'secure-connect-assignment.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("assignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('sales_data')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")
# Define the path to the CSV file
csv_file_path = 'sales_100.csv'  # Change this to your actual file path

# Prepare the insert statement with '?' placeholders
insert_statement = """
    INSERT INTO gold_sales_country (country, total_profit, total_revenue)
    VALUES (?, ?, ?)
"""

# Use a prepared statement for better performance and security
prepared = session.prepare(insert_statement)

# Open the CSV file and read the data
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        # Extract relevant columns from the CSV
        country = row['Country']
        total_profit = float(row['TotalProfit'])
        total_revenue = float(row['TotalRevenue'])

        # Execute the insert statement
        session.execute(prepared, (country, total_profit, total_revenue))

# Close the session and connection
cluster.shutdown()

print("Data inserted successfully into gold_sales_country.")


4.0.0.6816
Data inserted successfully into gold_sales_country.
